In [16]:
import pandas as pd
import numpy as np
url_orcid = "https://raw.githubusercontent.com/ahsan2github/datavis/master/ORCID_migrations_2016_12_16_by_person.csv";
df_all = pd.read_csv(url_orcid);
#print df_all.head().to_string();
nrows_with_na, ncols_with_na = df_all.shape;
df = df_all.dropna(how='any');
nrows_no_na, ncols_no_na = df.shape;
#print (df.head(3).to_string() + "\n");
print ("No of rows with missing data {0}, % row with missing values  {1}\n".format(nrows_with_na-nrows_no_na, \
                                            (nrows_with_na-nrows_no_na)/np.double(nrows_with_na)*100.0));

No of rows with missing data 504790, % row with missing values  68.04319372609915



In [17]:
#get data only when phd was awarded in 2016
df_2016 = df[df.phd_year==2016];
# get the total no of countries where phd was awarded, for 2016 only
no_awarding_countries, = df_2016.phd_country.unique().shape; # no of rows of an ndarray
print("Total no. of countries that awarded PhD in 2016: {} \n".format(no_awarding_countries));
# get country wise phd number and keep only the column that makes sense after summation
country_wise_phd = df_2016.groupby('phd_country').sum()[['has_phd']];
total_phd_awarded_2016 = country_wise_phd['has_phd'].sum();
# get the total phd awarded in 2016
country_wise_phd_prcnt = pd.DataFrame(columns=['phd_country','phd_prcnt']);
country_wise_phd_prcnt['phd_prcnt'] = country_wise_phd['has_phd'].\
    apply(lambda x: x / total_phd_awarded_2016 * 100);
country_wise_phd_prcnt['phd_country'] = country_wise_phd_prcnt.index.tolist();
country_wise_phd_prcnt['alpha-3']= '';
url_slim3 = "https://raw.githubusercontent.com/ahsan2github/datavis/master/slim-3.csv";
country_codes3= pd.read_csv(url_slim3);
for x in country_wise_phd_prcnt.index.tolist():
    # get a single row as a data frame where condition is true
    y = country_codes3[country_codes3['alpha-2']==x];
    country_wise_phd_prcnt.at[x,'phd_country'] = y['name'].values[0];
    country_wise_phd_prcnt.at[x,'alpha-3'] = y['alpha-3'].values[0]; 

Total no. of countries that awarded PhD in 2016: 162 



In [ ]:
ply.sign_in('user_name', 'api_key');;

In [21]:

# plot world wide data on maps
import plotly.plotly as ply
import plotly
data = [ dict(
        type='choropleth',
        autocolorscale = True,
        locations = country_wise_phd_prcnt['alpha-3'],
        z = country_wise_phd_prcnt['phd_prcnt'].astype(float),
        text = country_wise_phd_prcnt['phd_country'],
        marker = dict(
            line = dict (
                color = 'rgb(252,180,180)',
                width = 2
            ) ),
        colorbar = dict(
            title = "% PhD Awarded")
        ) ]

layout = dict(
        title = 'Phd awarded in 2016 around the world<br> shown as percentage of world-wide total \
        <br>(Hover for breakdown)',
        geo = dict(
            #scope='world',
            projection=dict( type='Mercator' ),
            showcoastlines = True,
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             );
    
fig = dict( data=data, layout=layout );
ply.iplot( fig, filename='d3-cloropleth-map' )

In [19]:
# get continent wise breakdown
import plotly.graph_objs as go
url_alpha2_continent = "https://raw.githubusercontent.com/ahsan2github/datavis/master/country_alpha-2_continent.csv";
cntry_continent_code = pd.read_csv(url_alpha2_continent);
country_wise_phd_prcnt['continent'] = "NA";
for x in country_wise_phd_prcnt.index.values:
     tmp = cntry_continent_code[cntry_continent_code['alpha-2']==x];
     if(tmp.empty != True):
         country_wise_phd_prcnt.at[x, 'continent'] = tmp['continent'].values[0];        
if(country_wise_phd_prcnt.index.values.shape[0] != country_wise_phd_prcnt['continent'].values.shape[0]):
    print("Didn't get continent values for all countries!");
xx = country_wise_phd_prcnt['continent'].apply(lambda x: x== '').values;
indx = np.where(xx)[0]    ;
country_wise_phd_prcnt.dropna();
country_wise_phd_prcnt = country_wise_phd_prcnt.dropna();
country_wise_phd_prcnt = country_wise_phd_prcnt[country_wise_phd_prcnt.continent != 'NA'];
continent_wise_prcnt = country_wise_phd_prcnt.groupby('continent').sum();
#print(continent_wise_prcnt.to_string())
#print(country_wise_phd_prcnt.sort_values(by=['continent']).to_string())
labels = continent_wise_prcnt.index.values; 
values = continent_wise_prcnt['phd_prcnt'].values;
colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1'];
cont_phd = go.Pie(labels=labels, values=values,
               hoverinfo='label+values', textinfo=' ', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)));
ply.iplot([cont_phd], filename='continent_wise_phd_prcnt')